In [20]:
import pandas as pd
import glob
import librosa
import wave
import struct
import math
import os
from scipy import fromstring, int16

In [21]:
out_folder = "../data/THINKLET/cut_wav/"

# 一応既に同じ名前のディレクトリがないか確認。
file = os.path.exists(out_folder)

if file == False:
    #保存先のディレクトリの作成
    os.mkdir(out_folder)

#filenameに読み込むファイル、timeにカットする間隔
def cut_wav(filename,time):  
    # timeの単位は[sec]

    # ファイルを読み出し
    wavf = filename + '.wav'
    wr = wave.open(wavf, 'r')

    # waveファイルが持つ性質を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr 
    integer = math.floor(total_time*100) # 小数点以下切り捨て
    t = int(time*100)  # 秒数[sec]
    frames = int(ch * fr * t /100)
    num_cut = int(integer//t)
    # waveの実データを取得し、数値化
    data = wr.readframes(wr.getnframes())
    wr.close()
    X = np.frombuffer(data, dtype=int16)

    for i in range(num_cut):
        #print(i)
        # 出力データを生成
        outf = out_folder + str(i) + '.wav' 
        start_cut = int(i*frames)
        end_cut = int(i*frames + frames)
        #print(start_cut)
        #print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, 'w')
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()

f_name = "../data/THINKLET/wav/VID_20201117_141446"
cut_time = 0.5
cut_wav(f_name,float(cut_time))

True


In [19]:
list_ceps = []
list_wav_names = []

# normalのwavを処理
filelist = glob.glob('../data/THINKLET/cut_wav/*.wav')  # ワイルドカードでnormal_*.wavのリストを作成

for filename in filelist:
    y, sr = librosa.core.load(filename,sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    
    # 複数のローリングウィンドウでそれぞれ20次元のMFCCを得られるので、その平均をとる。
    ceps = mfcc.mean(axis=1)
    
    # リストに追加
    list_ceps.append(ceps)  # 20次元のMFCCを追加
    list_wav_names.append(filename.split("/")[-1]) # ファイル名を追加
    
df_ceps = pd.DataFrame(list_ceps,index=list_wav_names)  

df_ceps

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
136.wav,-568.147278,183.299011,-34.668259,10.304159,-52.866241,11.656114,-30.963026,-0.669560,-8.648701,-5.897547,-5.476080,-0.911507,0.347181,-10.023408,2.171129,-6.557251,2.863284,3.063527,8.144311,5.351915
516.wav,-193.880478,54.136894,-119.512733,38.098866,-58.241386,20.764269,-22.241116,-15.432521,-14.394420,6.906713,-9.605135,-0.656744,-9.310603,-8.172753,0.606606,-2.719140,9.457510,-8.321296,16.333042,3.827804
33.wav,-355.735413,169.190903,65.438080,50.737507,-17.864841,-3.140628,-3.179377,-6.291368,4.393278,-5.907949,-18.390078,-6.245348,-6.966146,-5.690126,-2.060826,-8.821519,-3.137595,0.476350,4.107913,-1.043264
178.wav,-467.404449,86.825447,-12.258680,20.697073,-50.762119,17.930132,-40.922497,-1.152549,-4.381716,10.227168,-9.732504,-5.640327,-4.075131,-6.265492,1.444991,-6.943049,8.364826,7.167038,12.679031,-4.375541
572.wav,-238.875381,47.842545,-116.968109,35.846909,-54.857517,9.536098,-6.092222,2.575204,-0.864617,0.457014,-2.460625,6.144980,-11.033257,-7.027265,-5.635169,-6.823824,6.345049,-1.567742,8.519118,-7.165344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118.wav,-347.995667,140.707123,30.939554,77.493759,-14.267391,23.703949,17.273705,6.358284,2.259796,-2.375602,-3.547754,-6.459322,-4.822619,-4.413657,1.692172,-1.833112,-0.073515,-2.164999,-1.211606,-1.783897
44.wav,-488.791321,148.151489,26.161600,14.616423,-18.366205,0.833051,9.081454,-3.542047,6.058143,-2.213115,-7.943833,3.368388,2.669466,3.294552,7.862840,-1.687187,-7.031373,-3.022205,3.942701,1.338073
131.wav,-670.838745,141.935211,-15.789139,50.891876,-35.882412,14.163877,-16.987970,-7.013147,11.701699,-17.134478,-4.506948,-7.548579,9.998758,6.170135,0.868445,1.270908,5.750275,8.032962,-3.788675,3.652829
599.wav,-431.702576,43.950409,-50.985592,44.781178,-45.912197,11.951311,-7.504498,0.516879,2.797125,1.884424,-4.288806,6.053361,-1.564207,-7.305841,-3.224466,-5.447969,4.927634,-4.491106,5.351075,0.644402


In [22]:
df_ceps.to_csv("../data/wav.csv")